In [16]:
import re
words = open('../words/names.txt').read().splitlines()
words = [re.sub(r'[^a-z]','',word.lower()) for word in words]

In [17]:
characters = list('.abcdefghijklmnopqrstuvwxyz')
stoi = {s:i for i,s in enumerate(characters)}
itos = {i:s for s,i in stoi.items()}

In [18]:
import torch
import torch.nn.functional as F

In [19]:
xs = []
ys = []
for word in words:
    chs = ['.','.','.','.'] + list(word) + ['.']
    for ch1, ch2, ch3, ch4, ch5 in zip(chs, chs[1:], chs[2:], chs[3:], chs[4:]):
        ix1, ix2, ix3, ix4, ix5 = stoi[ch1], stoi[ch2], stoi[ch3], stoi[ch4], stoi[ch5]
        xs.append((ix1,ix2,ix3,ix4))
        ys.append(ix5)
        
xs = torch.tensor(xs)
ys = torch.tensor(ys)

g1 = torch.Generator().manual_seed(2147483647)
W = torch.randn((108,27), generator=g1, requires_grad=True)

In [20]:
from torch.utils.data import TensorDataset
dataset = TensorDataset(xs, ys)
g2 = torch.Generator().manual_seed(2147483647)
train_set, dev_set, test_set = torch.utils.data.random_split(dataset, [.8,.1,.1], generator=g2)

In [21]:
x_train = torch.stack([train_set[i][0]for i in range (len(train_set))])
y_train = torch.stack([train_set[i][1]for i in range (len(train_set))])

In [28]:
# training
for i in range (100):
    # forward
    logits = W[x_train[:, 0]]
    for i in range(1,4):
        logits += W[x_train[:, i] + 27*i]
    loss = F.cross_entropy(logits, y_train)
    # backward
    W.grad = None
    loss.backward()
    # update
    W.data += -5 * W.grad

In [29]:
loss.item()

2.4327328205108643

In [30]:
g = torch.Generator().manual_seed(2147483647)
for i in range(20):
    w = '....'
    while True:
        logits = torch.zeros(27)
        context = w[-4:]
        for i in range(4):
            ix = stoi[context[i]]
            logits += W[ix + i*27]
        counts = logits.exp()
        prob = counts / counts.sum(0, keepdims=True)
        ix = torch.multinomial(prob, 1, generator=g)
        char = itos[ix.item()]
        if char == '.':
            break
        w += char
    print(w[4:])

cexbbazom
surkiochity
well
imittaroraggak
kanda
sear
publtos
miotas
moriee
toog
wotedowka
havthssde
enkavglry
fobsp
hinide
vtarlasc
dsdrv
lewilep
awaist
pinp
